In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
.master("local[*]") \
.appName("NYC-TAXI-EDA") \
.getOrCreate()

In [3]:
green_df = spark.read.parquet("../data/2024/green/green_tripdata_2024-01.parquet")
green_df.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2024-01-01 00:46:55|  2024-01-01 00:58:25|                 N|         1|         236|         239|              1|         1.98|       12.8|  1.0|    0.

In [4]:
green_df.count()

56551

In [5]:
green_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
yellow_df = spark.read.parquet("../data/2024/yellow/yellow_tripdata_2024-01.parquet")
yellow_df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [7]:
yellow_df.count()

2964624

In [8]:
yellow_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



| Cột Green Taxi           | Cột Yellow Taxi          | Giải thích | Ghi chú khác biệt |
|--------------------------|---------------------------|------------|-------------------|
| VendorID                 | VendorID                  | ID nhà cung cấp taxi. | |
| lpep_pickup_datetime     | tpep_pickup_datetime      | Thời gian bắt đầu chuyến đi. | |
| lpep_dropoff_datetime    | tpep_dropoff_datetime     | Thời gian kết thúc chuyến đi. | |
| store_and_fwd_flag       | store_and_fwd_flag        | Cờ cho biết dữ liệu được lưu tạm rồi gửi (`Y`) hay gửi ngay (`N`). | |
| RatecodeID               | RatecodeID                | Mã định giá (1 = Chuẩn, 2 = JFK, 3 = Newark,...). | |
| PULocationID             | PULocationID              | ID vùng đón khách (TLC zone). | |
| DOLocationID             | DOLocationID              | ID vùng trả khách. | |
| passenger_count          | passenger_count           | Số hành khách. | |
| trip_distance            | trip_distance             | Quãng đường di chuyển (dặm). | |
| fare_amount              | fare_amount               | Cước phí cơ bản. | |
| extra                    | extra                     | Phụ phí thêm (giờ cao điểm, đêm khuya,...). | |
| mta_tax                  | mta_tax                   | Thuế MTA (thường $0.50). | |
| tip_amount               | tip_amount                | Tiền boa hành khách trả. | |
| tolls_amount             | tolls_amount              | Phí cầu đường. | |
| ehail_fee                | *(Không có)*              | Phí gọi xe qua app. | 🟩 **Chỉ Green Taxi có** do hoạt động vùng ngoại ô, thường gọi qua app. |
| improvement_surcharge    | improvement_surcharge     | Phụ phí cải tiến xe ($0.30). | |
| total_amount             | total_amount              | Tổng số tiền thanh toán. | |
| payment_type             | payment_type              | Hình thức thanh toán (1 = tiền mặt, 2 = thẻ,...). | |
| trip_type                | *(Không có)*              | 1 = Đón ngoài đường, 2 = Gọi qua app. | 🟩 **Chỉ Green Taxi có** – hỗ trợ dispatch. |
| congestion_surcharge     | congestion_surcharge      | Phí ùn tắc giao thông ($2.50 hoặc $2.75). | |
| cbd_congestion_fee       | cbd_congestion_fee        | Phí tắc nghẽn khu trung tâm Manhattan (CBD). | |
| *(Không có)*             | Airport_fee               | Phí sân bay (thường ở JFK). | 🟨 **Chỉ Yellow Taxi có** – thường phục vụ tuyến sân bay. |
